In [ ]:
import pandas as pd
import json

df = pd.read_json("data/processed/cases.json")

# Buat mapping case_id → amar_putusan
case_solutions = {
    row["case_id"]: row["amar_putusan"]
    for _, row in df.iterrows()
}

In [ ]:
from collections import Counter

def predict_outcome(query: str, k: int = 5) -> str:
    top_k = retrieve(query, k=k)
    solutions = [case_solutions.get(c, "") for c in top_k]

    # Majority vote
    most_common = Counter(solutions).most_common(1)
    predicted_solution = most_common[0][0] if most_common else ""

    return predicted_solution

In [ ]:
import pandas as pd
import json
import os

# Load queries
with open("data/eval/queries.json") as f:
    test_queries = json.load(f)

# Jalankan prediksi
results = []
for q in test_queries:
    pred_sol = predict_outcome(q['text'])
    top_k = retrieve(q['text'], k=5)

    results.append({
        "query_id": q['query_id'],
        "predicted_solution": pred_sol,
        "top_5_case_ids": top_k
    })

# Simpan ke CSV
df_preds = pd.DataFrame(results)

# Buat folder 'data/results' jika belum ada
os.makedirs("data/results", exist_ok=True)
df_preds.to_csv("data/results/predictions.csv", index=False)

In [ ]:
# perbandingan prediksi dan putusan sebenarnya
for q in test_queries[:5]:
    query = q["text"]
    predicted = predict_outcome(query)
    true_case_id = q["true_case_id"]
    true_solution = case_solutions[true_case_id]

    print(f"\nQuery ID       : {q['query_id']}")
    print(f"Predicted      : {predicted}")
    print(f"Ground Truth   : {true_solution}")



Query ID       : 0
Predicted      : menyatakan terdakwa mustaqim, s.t. bin muslimin hag (alm) tersebutdi atas, tidak terbukti secara sah dan meyakinkan bersalah melakukan tindakpidana korupsi secara bersama-sama dan berlanjut sebagaimana dalamdakwaan primair. Selain itu, menyatakan terdakwa dibebaskan dari dakwaan primair tersebut. Selanjutnya, menyatakan terdakwa mustaqim, s.t. bin muslimin hag (alm) tersebutdi atas, telah terbukti secara sah dan meyakinkan bersalah melakukan tindakpidana korupsi secara bersama-sama dan berlanjut sebagaimana dalamdakwaan subsidair. Terakhir, menjatuhkan pidana terhadap terdakwa oleh karena itu dengan pidanapenjara selama 2 (dua) tahun dan 6 (enam) bulan serta denda sejumlah rp100. 000. 000,00 (seratus juta rupiah) dengan ketentuan apabila denda tidakdibayar dapat diganti dengan pidana kurungan selama 3 (tiga) bulan.
Ground Truth   : menyatakan terdakwa ulis dewi purwanti,s.pd. terbukti secara sah danmeyakinkan bersalah melakukan “tindak pidana korups